In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, udf
from pyspark.sql.types import DoubleType

# --- Step 0: Ensure correct Python is used ---
os.environ["PYSPARK_PYTHON"] = "python"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python"

# --- Step 1: Initialize Spark ---
spark = SparkSession.builder.appName("Lab3_Task2_Similarity").getOrCreate()

# --- Step 2: Input Data ---
data = [
    (1, "alice smith"),
    (2, "bob smith"),
    (3, "charlie johnson"),
    (4, "david johnson"),
    (5, "alice smith")
]
df = spark.createDataFrame(data, ["id", "clean_name"])

# Tokenize names
df = df.withColumn("tokens", split(col("clean_name"), "\\s+"))

print("✅ Input Data:")
df.show(truncate=False)

# --- Step 3: Jaccard Similarity Function ---
def jaccard_similarity(tokens1, tokens2):
    try:
        set1, set2 = set(tokens1), set(tokens2)
        if not set1 or not set2:
            return 0.0
        return float(len(set1 & set2)) / len(set1 | set2)
    except Exception:
        return 0.0

jaccard_udf = udf(jaccard_similarity, DoubleType())

# --- Step 4: Compute Similarity for All Pairs ---
df_pairs = (
    df.alias("a")
    .crossJoin(df.alias("b"))
    .filter(col("a.id") < col("b.id"))   # avoid self-comparison
    .withColumn("similarity", jaccard_udf(col("a.tokens"), col("b.tokens")))
)

print("✅ Similarity Scores:")
df_pairs.show(truncate=False)

# --- Step 5: Stop Spark ---
spark.stop()


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 18 in stage 14.0 failed 1 times, most recent failure: Lost task 18.0 in stage 14.0 (TID 150) (Siri executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Users\subba\anaconda3\envs\bigdataenv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\worker.py", line 810, in main
  File "C:\Users\subba\anaconda3\envs\bigdataenv\Lib\site-packages\pyspark\python\lib\pyspark.zip\pyspark\serializers.py", line 594, in read_int
    length = stream.read(4)
  File "C:\Users\subba\anaconda3\envs\bigdataenv\lib\socket.py", line 717, in readinto
    return self._sock.recv_into(b)
TimeoutError: timed out

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read